# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,78.93,71,8,21.41,MU,1714919072
1,1,callao,-12.0667,-77.1500,76.71,88,100,4.61,PE,1714919072
2,2,broome,42.2506,-75.8330,50.90,94,100,11.50,US,1714919072
3,3,jiwani,25.0500,61.7417,85.19,61,0,23.04,PK,1714919059
4,4,papatowai,-46.5619,169.4708,43.14,89,80,9.51,NZ,1714919072


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size ="Humidity",
    scale =2,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
humidity_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

filtered_df = city_data_df[
    (city_data_df['Max Temp (F)'] >= 70) & 
    (city_data_df['Max Temp (F)'] <= 80) & 
    (city_data_df['Humidity'] < 70) &  # Adjust this threshold for low humidity
    (city_data_df['Cloudiness'] < 40) &  # Adjust this threshold for low cloudiness
    (city_data_df['Wind Speed'] < 20)  # Adjust this threshold for low wind speed
]

# Drop any rows with null values
# YOUR CODE HERE
filtered_df=filtered_df.dropna()

# Display sample data

# YOUR CODE HERE
filtered_df

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
45,45,toliara,-23.3500,43.6667,79.02,64,0,8.81,MG,1714919074
62,62,port elizabeth,-33.9180,25.5701,76.32,62,0,6.91,ZA,1714919074
107,107,hermanus,-34.4187,19.2345,71.80,49,12,18.90,ZA,1714918999
112,112,kaka,37.3482,59.6143,79.65,37,9,11.77,TM,1714919077
132,132,tan-tan,28.4380,-11.1032,73.35,60,5,18.41,MA,1714919078
133,133,richards bay,-28.7830,32.0377,78.30,67,0,4.50,ZA,1714918871
180,180,tolanaro,-25.0319,46.9987,75.27,64,20,13.80,MG,1714919080
183,183,marsh harbour,26.5412,-77.0636,76.03,64,4,15.28,BS,1714919080
207,207,namibe,-15.1961,12.1522,79.72,60,2,10.33,AO,1714919081
213,213,hotan,37.0997,79.9269,79.16,11,18,11.50,CN,1714919082


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df=filtered_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

hotel_df
# Display sample data


,City,Country,Lat,Lng,Humidity,Hotel Name
45,toliara,MG,-23.3500,43.6667,64,
62,port elizabeth,ZA,-33.9180,25.5701,62,
107,hermanus,ZA,-34.4187,19.2345,49,
112,kaka,TM,37.3482,59.6143,37,
132,tan-tan,MA,28.4380,-11.1032,60,
133,richards bay,ZA,-28.7830,32.0377,67,
180,tolanaro,MG,-25.0319,46.9987,64,
183,marsh harbour,BS,26.5412,-77.0636,64,
207,namibe,AO,-15.1961,12.1522,60,
213,hotan,CN,37.0997,79.9269,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
toliara - nearest hotel: Ambary
port elizabeth - nearest hotel: Waterford Hotel
hermanus - nearest hotel: Aloe guest house
kaka - nearest hotel: No hotel found
tan-tan - nearest hotel: Sable d´or
richards bay - nearest hotel: SUN1 Richards Bay
tolanaro - nearest hotel: Hôtel Mahavokey
marsh harbour - nearest hotel: Lofty Fig Villas
namibe - nearest hotel: Nelsal Pensao
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
saint-philippe - nearest hotel: Le Baril
archangelos - nearest hotel: Semina
bubaque - nearest hotel: Chez Julio
tsiombe - nearest hotel: No hotel found
cavalaire-sur-mer - nearest hotel: Hôtel La Pergola
aral - nearest hotel: No hotel found
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
balkanabat - nearest hotel: Nebitçi
korla - nearest hotel: Silver Star Hotel
serdar - nearest hotel: Wepa
east london - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
saint-pierre - nearest hotel: Tropic Hotel
chivhu 

,City,Country,Lat,Lng,Humidity,Hotel Name
45,toliara,MG,-23.3500,43.6667,64,Ambary
62,port elizabeth,ZA,-33.9180,25.5701,62,Waterford Hotel
107,hermanus,ZA,-34.4187,19.2345,49,Aloe guest house
112,kaka,TM,37.3482,59.6143,37,No hotel found
132,tan-tan,MA,28.4380,-11.1032,60,Sable d´or
133,richards bay,ZA,-28.7830,32.0377,67,SUN1 Richards Bay
180,tolanaro,MG,-25.0319,46.9987,64,Hôtel Mahavokey
183,marsh harbour,BS,26.5412,-77.0636,64,Lofty Fig Villas
207,namibe,AO,-15.1961,12.1522,60,Nelsal Pensao
213,hotan,CN,37.0997,79.9269,11,Hua Yi International Hotel (accepts foreigners)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map plot
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)